# Batching multiple event frames
To decrease the time our GPU waits for new data and sits idle, we'll increase the batch size next. Event recordings all have different length, even if it's just microseconds that they are off. In a mini-batch, all the tensors must have the same size. That is why we'll make use of a helper __collate__ function that pads tensors with zeros so that all the (transformed) recordings in the batch have the same shape. You will need to use the ToFrame transform for this to work.

In [1]:
import tonic
import tonic.transforms as transforms
import torch
from torch.utils.data import DataLoader
torch.manual_seed(1234)

sensor_size = tonic.datasets.NMNIST.sensor_size
frame_transform = transforms.ToFrame(sensor_size=sensor_size, time_window=10000)

dataset = tonic.datasets.NMNIST(save_to='./data', transform=frame_transform)

dataloader_batched = DataLoader(dataset, 
                                shuffle=True, 
                                batch_size=10, 
                                collate_fn=tonic.collation.PadTensors(batch_first=False))

events, targets = next(iter(dataloader_batched))

  0%|          | 0/1181572961 [00:00<?, ?it/s]

Extracting ./data/NMNIST/nmnist-archive.zip to ./data/NMNIST


By default, the resulting tensor will be in the format (Time, batch, channel, height, width) as in [PyTorch RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html?highlight=rnn#torch.nn.RNN).

In [2]:
events.shape

torch.Size([30, 10, 2, 34, 34])

In [3]:
targets

tensor([1, 4, 0, 3, 5, 0, 4, 7, 2, 4])

We can set `batch_first=True` in our collate class to change this behaviour.

In [4]:
dataloader_batched = DataLoader(dataset, 
                                shuffle=True, 
                                batch_size=10, 
                                collate_fn=tonic.collation.PadTensors(batch_first=True))

events, targets = next(iter(dataloader_batched))

In [5]:
events.shape

torch.Size([10, 31, 2, 34, 34])